In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data paths
train_data_dir = 'NewData/Train'
valid_data_dir = 'NewData/Test'

# Data preprocessing
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(train_data_dir, target_size=(224, 224), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_data_dir, target_size=(224, 224), batch_size=32, class_mode='binary')

# Model selection (Using a pre-trained ResNet50)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Freeze the pre-trained layers
base_model.trainable = False

# Model compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Training (fine-tuning only the top layers)
model.fit(train_generator, epochs=10, validation_data=valid_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc}')


Found 531 images belonging to 2 classes.
Found 177 images belonging to 2 classes.



Epoch 1/10


17/17 [==============================] - 73s 4s/step - loss: 0.5829 - accuracy: 0.7213 - val_loss: 0.5237 - val_accuracy: 0.7684
Epoch 2/10
17/17 [==============================] - 71s 4s/step - loss: 0.5684 - accuracy: 0.7213 - val_loss: 0.5113 - val_accuracy: 0.7684
Epoch 3/10
17/17 [==============================] - 71s 4s/step - loss: 0.5544 - accuracy: 0.7213 - val_loss: 0.5301 - val_accuracy: 0.7684
Epoch 4/10
17/17 [==============================] - 71s 4s/step - loss: 0.5543 - accuracy: 0.7213 - val_loss: 0.5027 - val_accuracy: 0.7684
Epoch 5/10
17/17 [==============================] - 71s 4s/step - loss: 0.5426 - accuracy: 0.7232 - val_loss: 0.5129 - val_accuracy: 0.7684
Epoch 6/10
17/17 [==============================] - 80s 5s/step - loss: 0.5347 - accuracy: 0.7213 - val_loss: 0.5044 - val_accuracy: 0.7684
Epoch 7/10
17/17 [==============================] - 84s 5s/step - loss: 

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data paths
train_data_dir = 'NewData/Train'
valid_data_dir = 'NewData/Test'

# Data preprocessing
# Create an instance of the ImageDataGenerator class with the desired transformations
train_datagen = ImageDataGenerator(
    rescale=1./255, # Rescale the pixel values to [0,1] range
    rotation_range=40, # Randomly rotate the images by 40 degrees
    width_shift_range=0.2, # Randomly shift the images horizontally by 20% of the width
    height_shift_range=0.2, # Randomly shift the images vertically by 20% of the height
    shear_range=0.2, # Randomly shear the images by 20 degrees
    zoom_range=0.2, # Randomly zoom the images by 20%
    horizontal_flip=True, # Randomly flip the images horizontally
    fill_mode='nearest' # Fill the missing pixels with the nearest filled value
)

# Create another instance of the ImageDataGenerator class for the validation data
# Only rescale the pixel values, no other transformations
valid_datagen = ImageDataGenerator(rescale=1./255)

# Create an iterator that can load the images from the directory and apply the transformations
train_generator = train_datagen.flow_from_directory(
    train_data_dir, # The directory where the training images are stored
    target_size=(224, 224), # The target size of the images, 224x224 pixels
    batch_size=32, # The batch size, 32 images per batch
    class_mode='binary' # The class mode, binary labels
)

# Create another iterator for the validation data
# No transformations, only rescaling
valid_generator = valid_datagen.flow_from_directory(
    valid_data_dir, # The directory where the validation images are stored
    target_size=(224, 224), # The target size of the images, 224x224 pixels
    batch_size=32, # The batch size, 32 images per batch
    class_mode='binary' # The class mode, binary labels
)

# Model selection (Using a pre-trained ResNet50)
base_model = tf.keras.applications.ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# Unfreeze the last convolutional block of the base model
for layer in base_model.layers:
    # Check if the layer name starts with 'conv5'
    if layer.name.startswith('conv5'):
        # Set the layer to be trainable
        layer.trainable = True
    else:
        # Keep the rest of the layers frozen
        layer.trainable = False

# Model compilation
# Use a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

# Training (fine-tuning the unfrozen layers)
# Use the train_generator and the valid_generator as the data sources
model.fit(train_generator, epochs=10, validation_data=valid_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(valid_generator)
print(f'Test accuracy: {test_acc}')

Found 324 images belonging to 2 classes.
Found 177 images belonging to 2 classes.
Epoch 1/10
11/11 [==============================] - 72s 6s/step - loss: 0.6303 - accuracy: 0.6358 - val_loss: 0.6194 - val_accuracy: 0.6836
Epoch 2/10
11/11 [==============================] - 64s 6s/step - loss: 0.5660 - accuracy: 0.6852 - val_loss: 0.5459 - val_accuracy: 0.7684
Epoch 3/10
11/11 [==============================] - 72s 7s/step - loss: 0.4196 - accuracy: 0.7994 - val_loss: 0.5062 - val_accuracy: 0.7684
Epoch 4/10
11/11 [==============================] - 68s 6s/step - loss: 0.4969 - accuracy: 0.7562 - val_loss: 0.5495 - val_accuracy: 0.7684
Epoch 5/10
11/11 [==============================] - 67s 6s/step - loss: 0.5020 - accuracy: 0.7469 - val_loss: 0.5618 - val_accuracy: 0.7684
Epoch 6/10
11/11 [==============================] - 74s 6s/step - loss: 0.5097 - accuracy: 0.7469 - val_loss: 0.5027 - val_accuracy: 0.7684
Epoch 7/10
11/11 [==============================] - 67s 6s/step - loss: 0.4399

In [7]:
model.save('trained_model_acc_87.h5')

C:\Users\Mysterious Soul\AppData\Roaming\Python\Python311\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [10]:
# Import the necessary modules
import numpy as np
from keras.models import load_model
from keras.preprocessing.image import load_img, img_to_array

# Load the model
model = load_model('trained_model_acc_87.h5')

# Load and preprocess the image
img = load_img('NewData/Valid/DairyMilk/cadbury-9-_jpg.rf.b2451da6e2debc82fb4f306f40f82580.jpg', target_size=(224, 224)) # Load the image and resize it
img = img_to_array(img) # Convert the image to a numpy array
img = img / 255. # Rescale the pixel values to [0, 1] range
img = np.expand_dims(img, axis=0) # Add a batch dimension

# Predict the label of the image
prob = model.predict(img) # Get the probability of the image belonging to the dairymilk class
prob = np.squeeze(prob) # Remove the singleton dimensions
label = np.round(prob) # Round the probability to the nearest integer

# Print the result
if label == 1:
    print(f'The image is dairymilk with a probability of {prob}.')
else:
    print(f'The image is not dairymilk with a probability of {1 - prob}.')


1/1 [==============================] - 4s 4s/step
The image is dairymilk with a probability of 0.9327480792999268.
